In [91]:
import pandas as pd
import numpy as np

In [92]:
df_games = pd.read_parquet('./src/cleaned/games.parquet')
df_items = pd.read_parquet('./src/cleaned/items.parquet')
df_reviews = pd.read_parquet('./src/cleaned/reviews.parquet')

In [93]:
#Agrupamos por item_id y sumamos el playtime_forever, luego lo unimos al df games
df_hours_for_item = df_items.groupby('item_id').agg({'playtime_forever': 'sum'}).reset_index()
df_games_full = pd.merge(df_games, df_hours_for_item, left_on='id', right_on='item_id', how='left').drop('item_id', axis=1)

#Agrupamos por item_id y contamos los jugadores, luego lo unimos al df games
df_players_for_item = df_items.groupby('item_id').agg({'user_id': 'count'}).reset_index()
df_games_full = pd.merge(df_games, df_players_for_item, left_on='id', right_on='item_id', how='left').drop('item_id', axis=1)
df_games_full = df_games_full.rename(columns={'user_id': 'player_count'})
df_games_full['player_count'] = df_games_full['player_count'].fillna(0)


In [94]:
#Generamos variables dummies para los tags
dummies = df_games_full['tags'].str.replace('[', '').str.replace(']', '').str.replace("'", '').str.get_dummies(sep=', ')
df_games_full = pd.concat([df_games_full, dummies], axis=1).drop('tags', axis=1)

In [96]:
#Condicionamos la columna metascore e imputamos los valores faltantes al promedio
df_games_full.loc[(df_games_full['metascore'] == "None") | (df_games_full['metascore'] == "NA"), 'metascore'] = np.nan
df_games_full['metascore'] = df_games_full['metascore'].astype('float64')
df_games_full['metascore'] = df_games_full['metascore'].mean()


In [100]:
print(df_games_full.columns.tolist())

['genres', 'app_name', 'title', 'release_date', 'specs', 'price', 'id', 'metascore', 'player_count', '"1990s"', '"Beat em up"', '"Shoot Em Up"', '1980s', '2.5D', '2D', '2D Fighter', '3D Platformer', '3D Vision', '4 Player Local', '4X', '6DOF', 'Abstract', 'Action', 'Action RPG', 'Action-Adventure', 'Adventure', 'Agriculture', 'Aliens', 'Alternate History', 'America', 'Animation & Modeling', 'Anime', 'Arcade', 'Arena Shooter', 'Artificial Intelligence', 'Assassin', 'Asynchronous Multiplayer', 'Atmospheric', 'Audio Production', 'Base Building', 'Based On A Novel', 'Basketball', 'Batman', 'Benchmark', 'Blood', 'Board Game', 'Bowling', 'Building', 'Bullet Hell', 'Bullet Time', 'CRPG', 'Capitalism', 'Card Game', 'Cartoon', 'Cartoony', 'Casual', 'Character Action Game', 'Character Customization', 'Chess', 'Choices Matter', 'Choose Your Own Adventure', 'Cinematic', 'City Builder', 'Class-Based', 'Classic', 'Clicker', 'Co-op', 'Co-op Campaign', 'Cold War', 'Colorful', 'Comedy', 'Comic Book', '